In [ ]:
! pip install spacy
! pip install negspacy
! pip install spacy_stanza
! pip install scikit-learn
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
import stanza
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset
import pandas as pd
import numpy as np
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import random



In [2]:
import spacy
import stanza
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset
import pandas as pd
import numpy as np
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.svm import LinearSVC
import random

In [20]:
df_test_tweets = pd.read_csv('Tweets_EN_sentiment_test.csv')
df_train_tweets = pd.read_csv('Tweets_EN_sentiment_train.csv')

print("Test Tweets")
display(df_test_tweets.head())
print("\nTrain Tweets")
display(df_train_tweets.head())

Test Tweets


,tweet,text,class
0,1229614408,G Night world,pos
1,1229158792,Morning all just got home from taking my daug...,pos
2,1228398639,thanks for the follow - how are you connected ...,pos
3,1229411644,re: Home Depot...hey! maybe we'll see you ther...,pos
4,1230107823,tak jsem poprve v zivote napsala zpravicku na ...,pos



Train Tweets


,tweet,text,class
0,1229448571,"some count sheep, you count cats",pos
1,1230261573,We haven't been to Turning Stone in ages! Have...,pos
2,1228861415,What up,pos
3,1228489365,I'm good,pos
4,1229800235,"I know, wonder if USER is going to buy an apar...",pos


In [21]:
def print_results(correct_predictions, total_predictions, true_positive, false_positive, false_negative):
  accuracy = correct_predictions/total_predictions
  precision = true_positive/(true_positive+false_positive)
  recall = true_positive/(true_positive+false_negative)
  F = (2 * precision * recall) / (precision + recall)
  print("Accuracy: ",accuracy)
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("F-Measure: ", F)

In [22]:
df_train_tweets = df_train_tweets.dropna()
df_test_tweets = df_test_tweets.dropna()
text_test_list = df_test_tweets['text']
class_test_list = df_test_tweets['class']

def sentiment_analysis_tb(text_list, classification_list):
  correct_predictions = 0
  total_predictions = 0
  false_positive = 0
  false_negative = 0
  true_positive = 0
  for text, classification in zip(text_list, classification_list):
    textBlob = TextBlob(text)
    sentiment = textBlob.sentiment.polarity
    if(sentiment <0 and classification == "neg"):
      correct_predictions += 1
    elif(sentiment >=0 and classification == "neg"):
      false_positive +=1
    elif(sentiment >=0 and classification == "pos"):
      true_positive += 1
      correct_predictions += 1
    elif(sentiment <0 and classification == "pos"):
      false_negative += 1
    total_predictions += 1
  print_results(correct_predictions, total_predictions, true_positive, false_positive, false_negative)

sentiment_analysis_tb(text_test_list, class_test_list)


Accuracy:  0.7996171670360669
Precision:  0.8689063784799718
Recall:  0.8944377267230955
F-Measure:  0.8814872192099148


# 3. without negation handeling

In [8]:
lexname="https://raw.githubusercontent.com/fmmb/Text-Mining/main/data/NRC-lexicon.csv"
data = pd.read_csv(lexname, encoding="utf-8", index_col=["English"])

lex = (data['Positive']-data['Negative']).to_dict()

In [9]:
def sentiment(text):
    score = 0
    for w in text:
      if w in lex:
        score += lex.get(w, 0)
    if score >= 0:
        return "pos"
    else:
        return "neg"


In [12]:
def sentiment_lex(text_list, classification_list):
  correct_predictions = 0
  total_predictions = 0
  false_positive = 0
  false_negative = 0
  true_positive = 0
  for text, classification in zip(text_list, classification_list):
    if(sentiment(text) == "pos"):
      if(classification == "pos"):
        correct_predictions += 1
        true_positive += 1
      else:
        false_positive += 1
    else:
      if(classification == "neg"):
        correct_predictions += 1
      else:
        false_negative += 1
    total_predictions += 1
  print_results(correct_predictions, total_predictions, true_positive, false_positive, false_negative)



# with preprocessing

In [13]:
tweet_tokenizer = TweetTokenizer()
porter_stemmer = PorterStemmer()

def tokenize(sentence):
    tokens = []
    tokens = tweet_tokenizer.tokenize(sentence)
    return tokens


def stem(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


def preprocess(sentence, stemmer):
    tokens = tokenize(sentence)
    stems = stem(tokens, porter_stemmer)
    return stems


def corpus_preprocess(text_list, stemmer):
  preprocessed_text = []
  for text in text_list:
    text_string = " ".join(text)
    preprocessed_text.append(preprocess(text_string, stemmer))
  return preprocessed_text


preprocessed_list = corpus_preprocess(text_test_list,porter_stemmer)
sentiment_lex(preprocessed_list, class_test_list)
#sentiment_lex(corpus_preprocess(text_test_list, porter_stemmer), class_test_list)



Accuracy:  0.8331654241386258
Precision:  0.8331654241386258
Recall:  1.0
F-Measure:  0.9089909870301166


In [14]:
def remove_stop_words(sent):
  stop_words = set(stopwords.words('english'))
  word_tokens = tweet_tokenizer.tokenize(str(sent))
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  return filtered_sentence

def corpus_remove_stop_words(text_list):
  preprocessed_text = []
  for text in text_list:
    preprocessed_text.append(remove_stop_words(text))
  return preprocessed_text

corpus_wo_stopword = corpus_remove_stop_words(text_test_list)

In [15]:
sentiment_lex(corpus_wo_stopword, class_test_list)

Accuracy:  0.7843038484787427
Precision:  0.855882011380792
Recall:  0.8911729141475212
F-Measure:  0.8731710206741307


# Negation Handling

In [ ]:
nlp_model = spacy.load("en_core_web_sm")

In [ ]:
def add_negation_pol(sentence):
  doc = nlp_model(sentence)
  negation = False
  neg_tokens= []
  sum = 0
  polarity = 0

  for token in doc:
    word = token.text
    if token.dep_ == "neg":
      negation = True
    elif negation:
      neg_tokens.append("NOT_"+ word)
      negation = False
      polarity = lex.get(word, 0) * -1
    else:
      neg_tokens.append(word)
      polarity = lex.get(word, 0)
    sum += polarity
  return sum

def add_negation(sentence):
  doc = nlp_model(sentence)
  negation = False
  neg_tokens= []

  for token in doc:
    if token.dep_ == "neg":
      negation = True
    elif negation:
      neg_tokens.append("NOT_"+ token.text)
      negation = False
    else:
      neg_tokens.append(token.text)
  return neg_tokens


def polarity_with_negation(sentence):

  handled_tokens = add_negation(sentence)
  sum = 0
  for word in handled_tokens:
    if word.startswith('NOT_'):
      simplified_word = word.replace('NOT_', '')
      polarity = lex.get(simplified_word, 0) * -1
    else:
      polarity = lex.get(word, 0)
    sum += polarity

  if sum >= 0:
    return "pos"
  else:
    return "neg"

print(polarity_with_negation("I dont love maths byt i love portuguese!"))
add_negation_pol("I dont love maths byt i love portuguese!")

In [ ]:
def sentiment_lex_with_negation(text_list, class_list):
  correct_predictions = 0
  total_predictions = 0
  false_positive = 0
  false_negative = 0
  true_positive = 0
  for text, classification in zip(text_list, class_list):
    text_string = " ".join(text)
    if(add_negation_pol(text_string) >=0):
      if(classification == "pos"):
        correct_predictions += 1
        true_positive += 1
      else:
        false_positive += 1
    else:
      if(classification == "neg"):
        correct_predictions += 1
      else:
        false_negative += 1
    total_predictions += 1
  print_results(correct_predictions, total_predictions, true_positive, false_positive, false_negative)

sentiment_lex_with_negation(corpus_wo_stopword, class_test_list)


Accuracy:  0.7875277050171268
Precision:  0.8568284489748639
Recall:  0.8944377267230955
F-Measure:  0.875229249245696


# 4. Aprendizagem Automática

In [ ]:
text_test= df_test_tweets['text']
class_test = df_test_tweets['class']

text_train = df_train_tweets['text']
class_train = df_train_tweets['class']



vectorizer_tfidf = TfidfVectorizer()
train_X = vectorizer_tfidf.fit_transform(text_train)
test_X = vectorizer_tfidf.transform(text_test)
print("TRAIN SIZE:", train_X.shape)
print("TEST SIZE:", test_X.shape)


feature_names = vectorizer_tfidf.get_feature_names_out()
feature_names

TRAIN SIZE: (39701, 32761)
TEST SIZE: (9926, 32761)


array(['00', '000', '0007', ..., 'òthink', 'ôjuice', 'û5'], dtype=object)

bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_CV = CountVectorizer()
train_X_CV = vectorizer_CV.fit_transform(text_train)
test_X_CV = vectorizer_CV.transform(text_test)
print("TRAIN SIZE:", train_X_CV.shape)
print("TEST SIZE:", test_X_CV.shape)

feature = vectorizer_CV.get_feature_names_out()
feature

TRAIN SIZE: (39701, 32761)
TEST SIZE: (9926, 32761)


array(['00', '000', '0007', ..., 'òthink', 'ôjuice', 'û5'], dtype=object)

In [ ]:
nlp = spacy.load("en_core_web_sm")

def filter_pos(text):
    doc = nlp(text)
    filtered_words = [token.text for token in doc if token.pos_ in ["NOUN", "ADJ", "VERB"]]
    return " ".join(filtered_words)

text_train_filtered = [filter_pos(text) for text in text_train]

In [ ]:
text_test_filtered = [filter_pos(text) for text in text_test]

In [ ]:
text_X_filtered = vectorizer_tfidf.fit_transform(text_train_filtered)
train_X_filtered = vectorizer_tfidf.transform(text_test_filtered)

text_X_CV_filtered = vectorizer_CV.fit_transform(text_train_filtered)
train_X_CV_filtered = vectorizer_CV.transform(text_test_filtered)

Preprocessamento

In [ ]:
text_train_wo_stopword = corpus_remove_stop_words(text_train)
text_test_wo_stopword = corpus_remove_stop_words(text_test)

text_train_wo_stopword_filtered = corpus_remove_stop_words(text_train_filtered)
text_test_wo_stopword_filtered = corpus_remove_stop_words(text_test_filtered)


preprocessed_train = corpus_preprocess(text_train_wo_stopword, porter_stemmer)
preprocessed_test = corpus_preprocess(text_test_wo_stopword, porter_stemmer)

preprocessed_train_filtered = corpus_preprocess(text_train_wo_stopword_filtered, porter_stemmer)
preprocessed_test_filtered = corpus_preprocess(text_test_wo_stopword_filtered, porter_stemmer)


preprocessed_train = [' '.join(doc) for doc in preprocessed_train]
preprocessed_test = [' '.join(doc) for doc in preprocessed_test]

preprocessed_train_filtered = [' '.join(doc) for doc in preprocessed_train_filtered]
preprocessed_test_filtered = [' '.join(doc) for doc in preprocessed_test_filtered]


In [ ]:
train_X_preprocessed = vectorizer_tfidf.fit_transform(preprocessed_train)
test_X_preprocessed = vectorizer_tfidf.transform(preprocessed_test)

train_X_preprocessed_filtered = vectorizer_tfidf.fit_transform(preprocessed_train_filtered)
test_X_preprocessed_filtered = vectorizer_tfidf.transform(preprocessed_test_filtered)

train_X_preprocessed_CV = vectorizer_CV.fit_transform(preprocessed_train)
test_X_preprocessed_CV = vectorizer_CV.transform(preprocessed_test)

train_X_preprocessed_filtered_CV = vectorizer_CV.fit_transform(preprocessed_train_filtered)
test_X_preprocessed_filtered_CV = vectorizer_CV.transform(preprocessed_test_filtered)

linear svm com tfidf sem preprocessamento

In [ ]:
clf_svm = svm.SVC()
clf_linear_svm = LinearSVC()
clf_linear_svm.fit(train_X, class_train)
predictions_l_wo = clf_linear_svm.predict(test_X)
print(classification_report(class_test_list, predictions_l_wo))



              precision    recall  f1-score   support

         neg       0.62      0.39      0.48      1656
         pos       0.89      0.95      0.92      8270

    accuracy                           0.86      9926
   macro avg       0.75      0.67      0.70      9926
weighted avg       0.84      0.86      0.84      9926



svm linear tfidf com preprocessamento

In [ ]:
clf_linear_svm.fit(train_X_preprocessed, class_train)
predictions_l_w = clf_linear_svm.predict(test_X_preprocessed)
print(classification_report(class_test_list, predictions_l_w))

              precision    recall  f1-score   support

         neg       0.60      0.35      0.44      1656
         pos       0.88      0.95      0.91      8270

    accuracy                           0.85      9926
   macro avg       0.74      0.65      0.68      9926
weighted avg       0.83      0.85      0.84      9926



svm linear CV sem preprocessamento

In [ ]:
clf_linear_svm.fit(train_X_CV, class_train)
predictions_l_wo_CV = clf_linear_svm.predict(test_X_CV)
print(classification_report(class_test_list, predictions_l_wo_CV))

              precision    recall  f1-score   support

         neg       0.52      0.41      0.46      1656
         pos       0.89      0.92      0.91      8270

    accuracy                           0.84      9926
   macro avg       0.70      0.67      0.68      9926
weighted avg       0.83      0.84      0.83      9926



svm linear CV com preprocessamento

In [ ]:
clf_linear_svm.fit(train_X_preprocessed_CV, class_train)
predictions_l_w_CV = clf_linear_svm.predict(test_X_preprocessed_CV)
print(classification_report(class_test_list, predictions_l_w_CV))

              precision    recall  f1-score   support

         neg       0.51      0.38      0.43      1656
         pos       0.88      0.93      0.90      8270

    accuracy                           0.84      9926
   macro avg       0.70      0.65      0.67      9926
weighted avg       0.82      0.84      0.83      9926



# svm

svm com tfidf sem preprocessamento

In [ ]:
clf_svm.fit(train_X, class_train)
predictions_svm_wo = clf_svm.predict(test_X)
print(classification_report(class_test_list, predictions_svm_wo))

svm com preprocessamento

In [ ]:
clf_svm.fit(train_X_preprocessed, class_train)
predictions_svm_w = clf_svm.predict(test_X_preprocessed)
print(classification_report(class_test_list, predictions_svm_w))

svm cv sem preprocessamento

In [ ]:
clf_svm.fit(train_X_CV, class_train)
predictions_svm_wo_CV = clf_svm.predict(test_X_CV)
print(classification_report(class_test_list, predictions_svm_wo_CV))

svm cv com preprocessamento

In [ ]:
clf_svm.fit(train_X_preprocessed_CV, class_train)
predictions_svm_w_CV = clf_svm.predict(test_X_preprocessed_CV)
print(classification_report(class_test_list, predictions_svm_w_CV))

# Multinomial Naive Bayes

multinomial naive bayes wo preprocessing

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf_nb = MultinomialNB()
clf_nb.fit(train_X, class_train)
predictions_nb_wo = clf_nb.predict(test_X)
print(classification_report(class_test_list, predictions_nb_wo))


multinomial naive bayes with preprocessing

In [ ]:
clf_nb.fit(train_X_preprocessed, class_train)
predictions_nb_w = clf_nb.predict(test_X_preprocessed)
print(classification_report(class_test_list, predictions_nb_w))

multinomial naive bayes cv without preprocessing

In [ ]:
clf_nb.fit(train_X_CV, class_train)
predictions_nb_wo_CV = clf_nb.predict(test_X_CV)
print(classification_report(class_test_list, predictions_nb_wo_CV))

multinomial naive bayes cv with preprocessing

In [ ]:
clf_nb.fit(train_X_preprocessed_CV, class_train)
predictions_nb_w_CV = clf_nb.predict(test_X_preprocessed_CV)
print(classification_report(class_test_list, predictions_nb_w_CV))

# 5. Utilização de transformadores

In [ ]:
from transformers import pipeline

In [ ]:
def sentiment_transformers(text_list, class_list, pipeline_name):

  if pipeline_name == 'sentiment-analysis':
    sentiment_pipeline = pipeline("sentiment-analysis")
  else:
    sentiment_pipeline = pipeline("sentiment-analysis", model = pipeline_name)

  predictions = [sentiment_pipeline(text)[0] for text in text_list]
  correct_predictions = 0
  total_predictions = 0
  false_positive = 0
  false_negative = 0
  true_positive = 0
  for prediction, true_label in zip(predictions, class_list):

    predicted_label = prediction['label']

    if predicted_label == "POSITIVE" or predicted_label== "NEU" or predicted_label == "POS":
      if true_label == "pos":
        true_positive += 1
        correct_predictions += 1
      else:
        false_positive += 1
    else:
      if true_label == "pos":
        false_negative += 1
      else:
        correct_predictions += 1

    total_predictions += 1
  print_results(correct_predictions, total_predictions, true_positive, false_positive, false_negative)


text_test_list = [str(text) for text in text_test_list]


In [ ]:
print("Default Sentiment Analysis")
default_SA = sentiment_transformers(text_test_list, class_test_list, 'sentiment-analysis')

In [ ]:
print("BERTweet Sentiment Analysis")
tweet_SA = sentiment_transformers(text_test_list, class_test_list, 'finiteautomata/bertweet-base-sentiment-analysis')

Fine-tuning

In [ ]:
!pip install datasets
import torch
torch.cuda.is_available()

In [23]:
import pandas as pd
from datasets import Dataset

df_train = df_train_tweets
df_test = df_test_tweets
df_train['class'] = df_train['class'].map({'pos': 1, 'neg': 0})
df_test['class'] = df_test['class'].map({'pos': 1, 'neg': 0})
df_train = df_train.rename(columns={'class': 'labels'})
df_test = df_test.rename(columns={'class': 'labels'})

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


small_train_dataset = train_dataset.shuffle(seed=5).select(range(1500))
small_test_dataset = test_dataset.shuffle(seed=5).select(range(150))

In [16]:
from transformers import AutoTokenizer
from datasets import Features, Value
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
def preprocess(ds):
  return tokenizer(ds["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess, batched=True)
tokenized_test = small_test_dataset.map(preprocess, batched=True)



Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [18]:
from transformers import AutoModelForSequenceClassification

In [11]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00


In [12]:
import evaluate

def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
   load_precision = evaluate.load("precision")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
   print("Precision:", precision)
   print("Accuracy:", accuracy)
   print("F1:", f1)
   return {"accuracy": accuracy, "f1": f1}

In [13]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [25]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
models = ["distilbert-base-uncased", "roberta-base"]

for model_name in models:
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

  args = TrainingArguments(
    output_dir = f'./results/{model_name}',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    save_strategy = "epoch",
    push_to_hub=False,)



  trainer = Trainer(
    model=model,
    args= args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,)



  trainer.train()

  trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Precision: 0.8880597014925373
Accuracy: 0.8333333333333334
F1: 0.9049429657794676


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Precision: 0.86
Accuracy: 0.86
F1: 0.9247311827956989
